In [ ]:
import os
import numpy as np
import pandas as pd
import glob

import psycopg2
from tqdm.notebook import tqdm
tqdm.pandas()
from typing import List, Optional

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="white") 

In [ ]:
MIMIC_FOLDER = ""
conn = psycopg2.connect("user=username password=password dbname=mimiciv")

def build_query(
    table_name: str,
    column_names: Optional[List[str]] = None,
    conditions: Optional[List[str]] = None,
    limit: Optional[int] = None
):
    if column_names is not None:
        col_str = ",".join(column_names)
    else:
        col_str = "*"
    limit_str = ""
    if limit is not None:
        limit_str += f" LIMIT {limit}"
    condition_str = ""
    if conditions is not None:
        condition_str += (" WHERE " + " AND ".join(conditions))
    return f"SELECT {col_str} from {table_name}" + condition_str + limit_str


def run_query(query: str, preview: Optional[bool] = True, save_to: str = None, **kwargs):
    print("EXECUTING QUERY:", query)
    df = pd.read_sql_query(query, conn)
    if preview:
        display(df)
    if save_to is not None:
        print(f"Saved to {save_to}")
        df.to_csv(save_to, **kwargs)
    return df

In [ ]:
patients_df = pd.read_csv(f"{MIMIC_FOLDER}/mimiciv/2.2/hosp/patients.csv.gz")
hadms = pd.read_csv(f"{MIMIC_FOLDER}/mimiciv/2.2/hosp/admissions.csv.gz", low_memory=False)
icustay_df = pd.read_csv(f"{MIMIC_FOLDER}/mimiciv/2.2/icu/icustays.csv.gz")

print(icustay_df.columns)

# Take sample of 10000 subject ids
subject_ids = patients_df['subject_id'].drop_duplicates()

filtered_hadms = hadms[hadms['subject_id'].isin(subject_ids)]
filtered_hadms = filtered_hadms.merge(patients_df[['subject_id', 'gender']], on='subject_id', how='left')


In [ ]:
# Build cohort from MIMIC only: start with admissions (no external cohort load)
valid_admissions = filtered_hadms[
    filtered_hadms['admission_type'].fillna('').str.upper().isin(('EW EMER.', 'DIRECT EMER.', 'URGENT'))
].copy()
# Placeholder label; overwritten below by task-specific logic (los, lactate_threshold, in_icu_mortality)
valid_admissions['boolean_value'] = False

# ICU timing/LOS comes from the derived table
icu_admissions = run_query(
    build_query(
        "mimiciv_derived.icustay_detail",
        column_names=["hadm_id", "stay_id", "icu_intime", "icu_outtime"]
    ),
    preview=False
)
icu_admissions['icu_intime'] = pd.to_datetime(icu_admissions['icu_intime'])
icu_admissions['icu_outtime'] = pd.to_datetime(icu_admissions['icu_outtime'])

# Keep the first ICU stay for each hospital admission so we retain stay_id for ICU vitals joins
first_icu_by_hadm = (
    icu_admissions
    .dropna(subset=['hadm_id', 'stay_id', 'icu_intime'])
    .sort_values(['hadm_id', 'icu_intime'])
    .drop_duplicates('hadm_id', keep='first')
    [['hadm_id', 'stay_id', 'icu_intime', 'icu_outtime']]
    .reset_index(drop=True)
)

# ICU type (careunit) comes from the raw ICU table (already loaded as icustay_df)
careunit_by_stay = (
    icustay_df[['stay_id', 'first_careunit', 'last_careunit']]
    .drop_duplicates('stay_id')
    .reset_index(drop=True)
)
first_icu_by_hadm = first_icu_by_hadm.merge(careunit_by_stay, on='stay_id', how='left')

first_icu_by_hadm['icu_los_hours'] = (
    first_icu_by_hadm['icu_outtime'] - first_icu_by_hadm['icu_intime']
).dt.total_seconds() / 3600

# Parameter: hours after ICU intime at which we define prediction_time
PREDICTION_HOURS_AFTER_ICU = 48  # customize this (e.g., 24, 48, 72, ...)
OBSERVATION_TIME = 48  # hours to look back for observations
# Label definition for task 'mortality': 'in_icu_mortality' = death during ICU stay (after prediction_time); 'hospital' = any in-hospital death
MORTALITY_LABEL_DEFINITION = 'in_icu_mortality'  # or 'hospital'

# Merge ICU timing and set prediction time to PREDICTION_HOURS_AFTER_ICU after ICU admission
valid_admissions = valid_admissions.merge(first_icu_by_hadm, on='hadm_id', how='inner')
valid_admissions['icu_intime'] = pd.to_datetime(valid_admissions['icu_intime'])

# New prediction_time definition: PREDICTION_HOURS_AFTER_ICU hours after ICU intime
valid_admissions['prediction_time'] = valid_admissions['icu_intime'] + pd.Timedelta(hours=PREDICTION_HOURS_AFTER_ICU)

# Require at least PREDICTION_HOURS_AFTER_ICU of ICU stay and prediction within the hospital stay window
valid_admissions = valid_admissions[
    (valid_admissions['icu_los_hours'] >= PREDICTION_HOURS_AFTER_ICU) &
    (valid_admissions['prediction_time'] >= valid_admissions['admittime']) &
    (valid_admissions['prediction_time'] <= valid_admissions['dischtime'])
].reset_index(drop=True)

# If task is long ICU LOS ("los"), define label: 1 if ICU stay > 7 days
if task == 'los':
    valid_admissions['boolean_value'] = (valid_admissions['icu_los_hours'] > 7 * 24).astype(int)

# If task is lactate threshold, check if lactate > 2 in next 24h after prediction_time
if task == 'lactate_threshold':
    # Query lactate values from bg table
    bg_df = run_query(
        build_query("mimiciv_derived.bg", column_names=["hadm_id", "charttime", "lactate"]),
        preview=False
    )
    bg_df['charttime'] = pd.to_datetime(bg_df['charttime'])
    bg_df = bg_df.dropna(subset=['lactate', 'charttime'])
    
    # Merge with valid_admissions to get prediction_time
    bg_with_pred = bg_df.merge(
        valid_admissions[['hadm_id', 'prediction_time']],
        on='hadm_id',
        how='inner'
    )
    bg_with_pred['prediction_time'] = pd.to_datetime(bg_with_pred['prediction_time'])
    
    # Filter to measurements in [prediction_time, prediction_time + 24h]
    mask = (
        (bg_with_pred['charttime'] >= bg_with_pred['prediction_time']) &
        (bg_with_pred['charttime'] <= bg_with_pred['prediction_time'] + pd.Timedelta(hours=24))
    )
    lactate_future = bg_with_pred.loc[mask]
    
    # Keep only patients with at least one lactate measurement in the 24h window
    hadm_ids_with_lactate = set(lactate_future['hadm_id'].unique())
    valid_admissions = valid_admissions[valid_admissions['hadm_id'].isin(hadm_ids_with_lactate)].reset_index(drop=True)
    
    # For each hadm_id, label = 1 if any lactate > 2 in window, else 0
    lactate_label = (
        lactate_future
        .groupby('hadm_id')['lactate']
        .max()
        .gt(2)
        .astype(int)
        .rename('lactate_label')
        .reset_index()
    )

    # Merge label back and overwrite boolean_value
    valid_admissions = valid_admissions.merge(lactate_label, on='hadm_id', how='left')
    valid_admissions['boolean_value'] = valid_admissions['lactate_label'].fillna(0).astype(int)
    valid_admissions = valid_admissions.drop(columns=['lactate_label'])

# ICU type for the selected ICU stay
valid_admissions['icu_type'] = valid_admissions['first_careunit'].fillna(valid_admissions['last_careunit'])

# Drop ICU types with < 500 samples or missing ICU type
icu_counts = valid_admissions['icu_type'].value_counts()
valid_admissions = valid_admissions[
    valid_admissions['icu_type'].notna() &
    valid_admissions['icu_type'].isin(icu_counts[icu_counts >= 500].index)
]

# Operation type: first coded procedure in the admission
operation_df = run_query(
    """
    SELECT DISTINCT ON (p.hadm_id)
        p.hadm_id,
        dp.long_title AS operation_type
    FROM mimiciv_hosp.procedures_icd p
    LEFT JOIN mimiciv_hosp.d_icd_procedures dp
      ON p.icd_code = dp.icd_code
     AND p.icd_version = dp.icd_version
    WHERE p.hadm_id IS NOT NULL
    ORDER BY p.hadm_id, p.seq_num NULLS LAST
    """,
    preview=False
)

valid_admissions = valid_admissions.merge(operation_df, on='hadm_id', how='left')

# Ventilation status at prediction_time (from mimiciv_derived.ventilation; run concepts if missing)
vent_df = run_query(
    build_query("mimiciv_derived.ventilation", column_names=["stay_id", "starttime", "endtime", "ventilation_status"]),
    preview=False
)
vent_df['starttime'] = pd.to_datetime(vent_df['starttime'])
vent_df['endtime'] = pd.to_datetime(vent_df['endtime'])
adm_vent = valid_admissions[['hadm_id', 'stay_id', 'prediction_time']].merge(vent_df, on='stay_id', how='left')
adm_vent['prediction_time'] = pd.to_datetime(adm_vent['prediction_time'])
mask = (adm_vent['starttime'] <= adm_vent['prediction_time']) & (adm_vent['prediction_time'] <= adm_vent['endtime'])
vent_at_pred = adm_vent.loc[mask].drop_duplicates('hadm_id', keep='first')[['hadm_id', 'ventilation_status']]
valid_admissions = valid_admissions.merge(vent_at_pred, on='hadm_id', how='left')
# Impute missing ventilation_status as "None" (no missingness indicator will be created)
valid_admissions['ventilation_status'] = valid_admissions['ventilation_status'].fillna('None')

# Override label based on LABEL_DEFINITION
if task == 'mortality' and MORTALITY_LABEL_DEFINITION == 'in_icu_mortality':
    valid_admissions['deathtime'] = pd.to_datetime(valid_admissions['deathtime'])
    valid_admissions['icu_outtime'] = pd.to_datetime(valid_admissions['icu_outtime'])
    # Exclude patients who died before prediction_time (temporal leakage prevention)
    valid_admissions = valid_admissions[
        (valid_admissions['deathtime'].isna()) | 
        (valid_admissions['deathtime'] >= valid_admissions['prediction_time'])
    ].reset_index(drop=True)
    # True if death occurred during the first ICU stay AFTER prediction_time (between prediction_time and icu_outtime)
    valid_admissions['boolean_value'] = (
        valid_admissions['deathtime'].notna()
        & (valid_admissions['deathtime'] >= valid_admissions['prediction_time'])
        & (valid_admissions['deathtime'] <= valid_admissions['icu_outtime'])
    )
elif task == 'mortality' and MORTALITY_LABEL_DEFINITION == 'hospital':
    valid_admissions['deathtime'] = pd.to_datetime(valid_admissions['deathtime'])
    valid_admissions = valid_admissions[
        (valid_admissions['deathtime'].isna()) |
        (valid_admissions['deathtime'] >= valid_admissions['prediction_time'])
    ].reset_index(drop=True)
    valid_admissions['boolean_value'] = valid_admissions['deathtime'].notna()

print(valid_admissions.columns)

# Keep only the required columns, including ICU, operation, and ventilation.
valid_admissions = valid_admissions[[
    'subject_id', 'hadm_id', 'stay_id', 'boolean_value', 'prediction_time',
    'admittime', 'dischtime', 'race', 'insurance', 'gender',
    'icu_type', 'operation_type', 'ventilation_status'
]].reset_index(drop=True)
valid_admissions = valid_admissions.drop_duplicates('subject_id', keep='first')
print(valid_admissions['boolean_value'].value_counts(normalize=True))

In [ ]:
from statsmodels.stats.proportion import test_proportions_2indep 

all_hadms = set(valid_admissions.hadm_id.dropna().tolist())
all_stays = set(valid_admissions['stay_id'].dropna().tolist()) if 'stay_id' in valid_admissions.columns else set()

def extract_features(lab_df, admissions, columns_to_mean, time=OBSERVATION_TIME, join_col='hadm_id', combined_ids=None):
    if combined_ids is None:
        combined_ids = set(admissions[join_col].dropna().tolist())

    df = lab_df.merge(admissions[[join_col, 'prediction_time']], on=join_col, how='inner')

    # Some tables are event-level (with charttime), while others are already aggregated per stay/admission.
    if 'charttime' in df.columns:
        df['charttime'] = pd.to_datetime(df['charttime'])
        df['prediction_time'] = pd.to_datetime(df['prediction_time'])

        # Filter to keep only measurements where charttime is before prediction_time
        df_filtered = df[df['charttime'] < df['prediction_time']]
        df_filtered = df_filtered[df_filtered['charttime'] >= (df_filtered['prediction_time'] - pd.Timedelta(hours=time))]

        # Group by join key and keep the last measurement in the observation window
        df_filtered = df_filtered.sort_values(by=[join_col, 'charttime'])
        # last_df = df_filtered.groupby(join_col).tail(1)[[join_col] + columns_to_mean].reset_index(drop=True)
        last_vals = (
            df_filtered
            .groupby(join_col, as_index=False)[columns_to_mean]
            .agg(lambda s: s.dropna().iloc[-1] if s.notna().any() else np.nan)
        )

    else:
        # For pre-aggregated tables (e.g., first_day_vitalsign), keep one row per join key.
        last_df = df.sort_values(by=[join_col]).drop_duplicates(join_col, keep='last')[[join_col] + columns_to_mean].reset_index(drop=True)

    all_ids_df = pd.DataFrame({join_col: list(combined_ids)})
    # final_df = pd.merge(all_ids_df, last_df, on=join_col, how='left')
    final_df = pd.DataFrame({join_col: list(combined_ids)}).merge(last_vals, on=join_col, how='left')

    return final_df

def calculate_nan_rate(df, columns):
    nan_rates = {}
    
    # Calculate the NaN rate for each column
    for column in columns:
        nan_count = df[column].isna().sum()
        nan_rate = nan_count / len(df) * 100  # Percentage of NaNs
        nan_rates[column] = nan_rate
    
    return nan_rates

In [ ]:
clot_df = run_query(build_query("mimiciv_derived.coagulation", column_names=["*"]), preview=False)
columns_to_mean = ['inr','pt','fibrinogen']

filtered_clot_df = extract_features(clot_df, valid_admissions, columns_to_mean, time=OBSERVATION_TIME)
print(filtered_clot_df)
nan_rates = calculate_nan_rate(filtered_clot_df, columns_to_mean)
print(nan_rates)

cardiac_df = run_query(build_query("mimiciv_derived.cardiac_marker", column_names=["*"]), preview=False)
columns_to_mean = ['troponin_t']

filtered_trop_df = extract_features(cardiac_df, valid_admissions, columns_to_mean, time=OBSERVATION_TIME)
print(filtered_trop_df)
nan_rates = calculate_nan_rate(filtered_trop_df, columns_to_mean)
print(nan_rates)

In [ ]:
abg_df = run_query(build_query("mimiciv_derived.bg", column_names=["*"]), preview=False)
columns_to_mean = ["po2","pco2","ph","lactate","so2"]

filtered_abg_df = extract_features(abg_df, valid_admissions, columns_to_mean, time=OBSERVATION_TIME)
print(filtered_abg_df)
nan_rates = calculate_nan_rate(filtered_abg_df, columns_to_mean)
print(nan_rates)

In [ ]:
bmp_df = run_query(build_query("mimiciv_derived.chemistry", column_names=["hadm_id", "charttime", "bicarbonate","bun","creatinine","glucose"]), preview=False)
bmp_features = ["bicarbonate","bun","creatinine","glucose"]

filtered_bmp_df = extract_features(bmp_df, valid_admissions, bmp_features, time=OBSERVATION_TIME)
print(filtered_bmp_df)
nan_rates = calculate_nan_rate(filtered_bmp_df, bmp_features)
print(nan_rates)

In [ ]:
wbc_df = run_query(build_query("mimiciv_derived.blood_differential", column_names=["hadm_id", "charttime", "neutrophils_abs", "lymphocytes_abs"]), preview=False)
wbc_features = ["neutrophils_abs", "lymphocytes_abs"]

filtered_wbc_df = extract_features(wbc_df, valid_admissions, wbc_features, time=OBSERVATION_TIME)
print(filtered_wbc_df)
nan_rates = calculate_nan_rate(filtered_wbc_df, wbc_features)
print(nan_rates)


In [ ]:
# Add other baseline information for patient (age and icu related information)
demo_df = run_query(build_query("mimiciv_derived.age", column_names=["*"]), preview=False)

# Suspected infection time per stay (for later filter: suspected infection before prediction_time)
sepsis3_df = run_query(
    build_query("mimiciv_derived.sepsis3", column_names=["stay_id", "suspected_infection_time"]),
    preview=False
)
sepsis3_df["suspected_infection_time"] = pd.to_datetime(sepsis3_df["suspected_infection_time"])
valid_admissions = valid_admissions.merge(
    sepsis3_df.drop_duplicates("stay_id"),
    on="stay_id",
    how="left"
)

filtered_demo_df = valid_admissions.merge(demo_df[["hadm_id", "age"]], on="hadm_id", how="left")
#print(filtered_demo_df)

In [ ]:
vitals_df = run_query(build_query("mimiciv_derived.vitalsign", column_names=["stay_id", "charttime", "mbp", "heart_rate", "spo2"]), preview=False)
vitals_features = ["mbp", "heart_rate", "spo2"]

filtered_vitals_df = extract_features(vitals_df, valid_admissions, vitals_features, time=OBSERVATION_TIME, join_col='stay_id')
nan_rates = calculate_nan_rate(filtered_vitals_df, vitals_features)
print(nan_rates)

stay_map = valid_admissions[['hadm_id', 'stay_id']].drop_duplicates('stay_id')

filtered_vitals_df = (
    stay_map
    .merge(filtered_vitals_df, on='stay_id', how='left')
    .drop(columns=['stay_id'])
)

print(filtered_vitals_df.head())

In [ ]:
data_df = filtered_bmp_df.merge(filtered_abg_df, on='hadm_id', how='left')
data_df = data_df.merge(filtered_clot_df, on='hadm_id', how='left')
data_df = data_df.merge(filtered_trop_df, on='hadm_id', how='left')
data_df = data_df.merge(filtered_vitals_df, on='hadm_id', how='left')
data_df = data_df.merge(filtered_wbc_df, on='hadm_id', how='left')
data_df = data_df.merge(
    filtered_demo_df[['hadm_id', 'boolean_value', 'age', 'gender', 'icu_type', 'operation_type', 'ventilation_status', 'suspected_infection_time', 'prediction_time']],
    on='hadm_id',
    how='left'
)

need_columns = bmp_features + vitals_features
# Only keep rows that have at least one non-missing value among the lab features
data_df = data_df.dropna(subset=need_columns, how='all')

In [ ]:
llm_feature_mapping = {
    # --- Demographics & Target ---
    'age': 'Age',
    'gender': 'Gender',
    'icu_type': 'ICU Type',
    'operation_type': 'Operation Type',
    'ventilation_status': 'Ventilation Status',
    
    # --- Routine Vitals (Automatic / Low Discretion) ---
    'heart_rate': 'Heart Rate (BPM)',
    'map': 'Mean Arterial Pressure (MAP)',
    'spo2': 'Pulse Oximetry (SpO2)',
    
    # --- Metabolic Panel (Routine Venous Draw) ---
    'bicarbonate': 'Serum Bicarbonate (HCO3)',
    'bun': 'Blood Urea Nitrogen (BUN)',
    'creatinine': 'Serum Creatinine',
    'glucose': 'Serum Glucose',
    
    # --- Critical Labs (High Discretion / "The Signal") ---
    # "Serum" implies a specific order was placed
    'lactate': 'Serum Lactate', 
    'inr': 'INR (Coagulation)',
    'pt': 'Prothrombin Time (PT)',
    'fibrinogen': 'Fibrinogen',
    'troponin_t': 'Troponin T',
    'neutrophils_abs': 'Neutrophils Absolute Count',
    'lymphocytes_abs': 'Lymphocytes Absolute Count',
    
    # --- Arterial Blood Gas (Painful / High Discretion) ---
    # Renaming these is CRITICAL so the LLM knows they are from an Art Line/Stab
    # and not just a finger probe.
    'ph': 'Arterial pH',
    'po2': 'Arterial O2 Pressure (PaO2)',
    'pco2': 'Arterial CO2 Pressure (PaCO2)',
    'so2': 'Arterial O2 Saturation (SaO2)',
}

# Apply the renaming
# Assuming your dataframe is named 'data_df'
data_df.rename(columns=llm_feature_mapping, inplace=True)

In [ ]:
# Filter data_df to only MICU patients
micu_df = data_df[data_df['ICU Type'] == 'Medical Intensive Care Unit (MICU)']

# Compute prevalence of boolean_value for micu_df
if 'boolean_value' in micu_df.columns:
    prevalence = micu_df['boolean_value'].mean()
    print(f"Prevalence of boolean_value in MICU cohort: {prevalence:.3f}")
else:
    print("Column 'boolean_value' not found in micu_df.")

ccu_df = data_df[data_df['ICU Type'] == 'Coronary Care Unit (CCU)']

# Compute prevalence of boolean_value for ccu_df
if 'boolean_value' in ccu_df.columns:
    prevalence = ccu_df['boolean_value'].mean()
    print(f"Prevalence of boolean_value in CCU cohort: {prevalence:.3f}")
else:
    print("Column 'boolean_value' not found in ccu_df.")

# Remove 'operation type' column if present before saving
micu_save = micu_df.drop(columns=['operation type'], errors='ignore')
ccu_save = ccu_df.drop(columns=['operation type'], errors='ignore')